In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
#from nltk import 

In [2]:
# nltk

import nltk as nltk
# nltk.download('punkt')
# nltk.download('wordnet')
# nltk.download('stopwords')
from nltk.tokenize import word_tokenize
from nltk.tokenize import RegexpTokenizer
tokenizer = RegexpTokenizer(r'\w+')

from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
from nltk.corpus import stopwords
nltk_stopwords =  set(stopwords.words('english'))
Morz_stopwords = list( pd.read_csv('Conditioner/longstopwords.txt',header=None )[0]     )
# Collections
from collections import Counter

import pickle 


### Importing data

In [3]:
# sample_records = np.load('sample_abs.npz')
sample_abs_list = np.load('sample_abs_list.npz')
sample_abstract = sample_abs_list['abs_list'][1]

In [4]:
dictionary = pd.read_csv('Dictator/refinedDictionary.csv')
dictionary = dictionary.set_index('0').to_dict()['1']
len(dictionary)

5552

In [5]:
tokenizer.tokenize(sample_abstract )

['The',
 'issue',
 'is',
 'the',
 'expediency',
 'of',
 'the',
 'QCD',
 'notions',
 'use',
 'in',
 'the',
 'low',
 'energy',
 'region',
 'ndown',
 'to',
 'the',
 'confinement',
 'scale',
 'and',
 'in',
 'particular',
 'the',
 'efficacy',
 'of',
 'the',
 'QCD',
 'ninvariant',
 'coupling',
 'bar',
 'alpha',
 '_s',
 'Q',
 '2',
 'with',
 'a',
 'minimal',
 'analytic',
 'modification',
 'in',
 'nthis',
 'domain',
 'To',
 'this',
 'goal',
 'we',
 'overview',
 'a',
 'quite',
 'recent',
 'progress',
 'in',
 'application',
 'nof',
 'the',
 'ghost',
 'free',
 'Analytic',
 'Perturbative',
 'Theory',
 'approach',
 'with',
 'no',
 'adjustable',
 'nparameters',
 'for',
 'QCD',
 'in',
 'the',
 'region',
 'below',
 '1',
 'GeV',
 'Among',
 'them',
 'the',
 'Bethe',
 'Salpeter',
 'nanalysis',
 'of',
 'the',
 'meson',
 'spectra',
 'and',
 'spin',
 'dependent',
 'polarization',
 'Bjorken',
 'sum',
 'nrule',
 'The',
 'impression',
 'is',
 'that',
 'there',
 'is',
 'a',
 'chance',
 'for',
 'the',
 'theoretic

### Conditionor

In [6]:
"""
The objective of this function is to get an abstract and return 
a list of words in the abstract. 
The idea is to get rid of some non-neccessary words and turn 
everything into their roots, also get rid of the latex and numbers
and ...
This function is to be used for making the dictionary and also in 
the vectorizer. 
"""

def abstract_tokenizor(abstract):
#     abstract_tl =  word_tokenize(abstract) 
#     abstract_tl = [lemmatizer.lemmatize(i).lower() for i in word_tokenize(abstract) ]
    abstract_tl = [lemmatizer.lemmatize(i).lower() for i in tokenizer.tokenize(abstract) ]
    abstract_tl = [word for word in abstract_tl if 
                   (not word in nltk_stopwords) 
                   and (not word in Morz_stopwords)
                  and (word.isalpha() )]
    """
    1. Tokenize, => lowercase
    2. Lemetize
    3. get rid of stop words
    4. get rid of latex and extra commands
    
    """
    return abstract_tl
    

In [66]:
# [i for i in  Counter(  abstract_tokenizor(sample_abstract) ) ]

### Dictator

In [10]:
'''
Make a dictionary from all the abstract in the list
'''

def dictator(abs_list):
    word_list = []
    for i in abs_list:
        word_list += abstract_tokenizor(i)
        
    return dict( Counter(word_list) )

In [11]:
%%time
sample_dictionary = dictator( sample_abs_list['abs_list'][:500] )

Wall time: 890 ms


In [17]:
list(sample_dictionary).index('qcd')

4

### Vectorizor

In [59]:
'''
Takes an input abstract and returns the 
vector corresponding to that.
'''
def vectorizer(abstract, dictionary):
    abstract_word_list = abstract_tokenizor(abstract)
    dict_word_list = list(dictionary)
    abst_dict = Counter(abstract_word_list)
    word_count = len(abstract_word_list)
    abstract_vec= [ abst_dict[key] / word_count for key in dict_word_list ]
        
    return abstract_vec

In [71]:
def vectorizer(abstract, dictionary):
    abstract_word_list = abstract_tokenizor(abstract)
    abstract_vec = np.zeros(len(dictionary))
    dict_word_list = list(dictionary)
    word_count = len(abstract_word_list)
    for i in abstract_word_list:
        if i in dict_word_list:
            abstract_vec[dict_word_list.index(i) ] +=1
        
    return abstract_vec/sum(abstract_vec)

In [7]:
def vectorizer(abstract, dictionary):
#     Tokenize the abstract
    abstract_word_list = abstract_tokenizor(abstract)
#     Initiate abstract vector
    abstract_vec = np.zeros(len(dictionary))
#     Call the dictionary
    dict_word_list = list(dictionary)
    word_count = len(abstract_word_list)
#     Abstract words Frequency
    abst_dict = Counter(abstract_word_list)
    
    for i in abst_dict:
        if i in dict_word_list:
            abstract_vec[dict_word_list.index(i) ] += abst_dict[i]
        
    return abstract_vec/sum(abstract_vec)

In [8]:
sum( vectorizer(sample_abs_list['abs_list'][2] , dictionary) )

0.9999999999999996

In [12]:
%%time
[len(vectorizer(sample_abs_list['abs_list'][i], sample_dictionary) ) for i in range(30)]

Wall time: 10.2 s


[7696,
 7696,
 7696,
 7696,
 7696,
 7696,
 7696,
 7696,
 7696,
 7696,
 7696,
 7696,
 7696,
 7696,
 7696,
 7696,
 7696,
 7696,
 7696,
 7696,
 7696,
 7696,
 7696,
 7696,
 7696,
 7696,
 7696,
 7696,
 7696,
 7696]